In [ ]:
---
title: Assignment 03
author:
  - name: Mahira Ayub
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
    docx: default
    pdf: default
date-modified: today
date-format: long
engine: juptyer
embed-resourves: true
execute:
  echo: false
  eval: false
  freeze: auto
  juptyer: python3
---

## Load the Dataset
The instruction below provides you with general keywords for columns used in the lightcast file. See the data schema generated after the load dataset code above to use proper column name. For each visualization, customize colors, fonts, and styles to avoid a 2.5-point deduction. Also, provide a two-sentence explanation describing key insights drawn from the graph.

# Load the Raw Dataset**:
- Use Pyspark to the `lightcast_data.csv` file into a DataFrame.
- You can reuse the previous code.
- [Copying code from your friend constitutes plagiarism. DO NOT DO THIS! - bloodred-bold].

In [ ]:
#| eval: true
#| echo: true
#| fig-align: center

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "svg"
from pyspark.sql import SparkSession
import re
import numpy as np
import plotly.graph_objects as go
from pyspark.sql.functions import col, split, explode, regexp_replace, transform, when
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id

np.random.seed(42)

pio.renderers.default = "notebook"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
# print("--This is Diagnostic check, No need to print it in the final doc---")

# df.printSchema()   # comment this line when rendering the submission
# df.show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/22 00:57:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


TypeError: 'JavaPackage' object is not callable

In [13]:
from pyspark.sql import SparkSession

# Start a Spark session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load the CSV file into a Spark DataFrame
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("lightcast_job_postings.csv")

df.createOrReplaceTempView("jobs")

# Show schema
df.printSchema()

TypeError: 'JavaPackage' object is not callable